# August Perez Capstone Three Project:
## Characterization of High Accident Situations

Using neural networks / deep learning I plan on analyzing a conglomerate dataset to characterize what combined variables of road types, times, and conditions have the highest probability of a car accident occuring.

### Goal
Build models for characterization of locations with crash probability of >=80% (or top 10 if minimal quantity of >=80% crash probability)
and an analysis of times, days of the week, and dates of the year with crash probability >=60% for the locations characterized.

    Crash probability Thresholds chosen arbitrarily. For real-world application, each agency utilizing this needs to determine thresholds specific to their needs and capabilities.

### Data source:
Accidents in France from 2005 to 2016 (https://www.kaggle.com/datasets/ahmedlahlou/accidents-in-france-from-2005-to-2016/data)


    
#### About the dataset:
- A collection of 5 datasets pertaining to car crashes in France from 2005 to 2016
    - characteristics
        - Details about each crash
    - holidays
        - Dates from 2005 to 2016 that are holidays
    - places
        - Details about accident locations
    - users
        - Details about persons involved in the accident
    - vehicles
        - Details about vehicles involved in the accident

## Note on raw format cells:
Converted to raw format so the code within the cell won't execute. Kept the cell because it may have use for looking back on it in the future even if it did not have use within this project.

## Imports:

In [1]:
# data manipulation and math

import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import datetime as dt

# plotting and visualization

import matplotlib.pyplot as plt
import seaborn as sns

# modeling & pre-processing
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense

#warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
    #reminder code so I can ignore warnings for single code cells:
        # warnings.filterwarnings('ignore')
        # warnings.resetwarnings()

%matplotlib inline


### Set random seed for reproducability
Note that this should not be done for models used in real-world applications

In [2]:
np.random.seed(9)

## Load the data into a pandas df's

### Column Descriptions
Large section. Suggested to keep collapsed.

CARACTERISTICS :

**Num_Acc** : Accident ID

jour : Day of the accident

mois : Month of the accident

an : Year of the accident

hrmn : Time of the accident in hour and minutes (hhmm)

**lum** : Lighting : lighting conditions in which the accident occurred

    1 - Full day

    2 - Twilight or dawn

    3 - Night without public lighting

    4 - Night with public lighting not lit

    5 - Night with public lighting on

dep : Departmeent : INSEE Code (National Institute of Statistics and Economic Studies) of the departmeent followed
by a 0 (201 Corse-du-Sud - 202 Haute-Corse)

com : Municipality: The commune number is a code given by INSEE. The code has 3 numbers set to the right.

**Localisation (agg)** :

    1 - Out of agglomeration

    2 - In built-up areas

**int** : Type of Intersection :

    1 - Out of intersection

    2 - Intersection in X

    3 - Intersection in T

    4 - Intersection in Y

    5 - Intersection with more than 4 branches

    6 - Giratory

    7 - Place

    8 - Level crossing

    9 - Other intersection

**atm** : Atmospheric conditions:

    1 - Normal

    2 - Light rain

    3 - Heavy rain

    4 - Snow - hail

    5 - Fog - smoke

    6 - Strong wind - storm

    7 - Dazzling weather

    8 - Cloudy weather

    9 - Other

**col** : Type of collision:

    1 - Two vehicles - frontal

    2 - Two vehicles - from the rear

    3 - Two vehicles - by the side

    4 - Three vehicles and more - in chain

    5 - Three or more vehicles - multiple collisions

    6 - Other collision

    7 - Without collision

adr : Postal address: variable filled in for accidents occurring in built-up areas

gps : GPS coding: 1 originator character:

    M = Métropole

    A = Antilles (Martinique or Guadeloupe)

    G = Guyane

    R = Réunion

    Y = Mayotte

Geographic coordinates in decimal degrees:

    lat : Latitude

    long : Longitude

Places:

Num_Acc : Accident ID

**catr** : Category of road:

    1 - Highway

    2 - National Road

    3 - Departmental Road

    4 - Communal Way

    5 - Off public network

    6 - Parking lot open to public traffic

    9 - other

voie : Road Number

V1: Numeric index of the route number (example: 2 bis, 3 ter etc.)

V2: Letter alphanumeric index of the road

**circ**: Traffic regime:

    1 - One way

    2 - Bidirectional

    3 - Separated carriageways

    4 - With variable assignment channels

**nbv**: Total number of traffic lanes

**vosp**: Indicates the existence of a reserved lane, regardless of whether or not the accident occurs on that lane.

    1 - Bike path

    2 - Cycle Bank

    3 - Reserved channel

**Prof**: Longitudinal profile describes the gradient of the road at the accident site

    1 - Dish

    2 - Slope

    3 - Hilltop

    4- Hill bottom

pr: Home PR number (upstream terminal number)

pr1: Distance in meters to the PR (relative to the upstream terminal)

**plan**: Drawing in plan:

    1 - Straight part

    2 - Curved on the left

    3 - Curved right

    4 - In "S"

**lartpc**: Central solid land width (TPC) if there is

**larrout**: Width of the roadway assigned to vehicle traffic are not included the emergency stop strips,
CPRs and parking spaces

**surf**: surface condition

    1 - normal

    2 - wet

    3 - puddles

    4 - flooded

    5 - snow

    6 - mud

    7 - icy

    8 - fat - oil

    9 - other

**infra**: Development - Infrastructure:

    1 - Underground - tunnel

    2 - Bridge - autopont

    3 - Exchanger or connection brace

    4 - Railway

    5 - Carrefour arranged

    6 - Pedestrian area

    7 - Toll zone

**situ**: Situation of the accident:

    1 - On the road

    2 - On emergency stop band

    3 - On the verge

    4 - On the sidewalk

    5 - On bike path

env1: school point: near a school

USERS:

Acc_number: Accident identifier.

Num_Veh: Identification of the vehicle taken back for each user occupying this vehicle (including pedestrians who are
attached to the vehicles that hit them)

place: Allows to locate the place occupied in the vehicle by the user at the time of the accident

catu: User category:

    1 - Driver

    2 - Passenger

    3 - Pedestrian

    4 - Pedestrian in rollerblade or scooter

grav: Severity of the accident: The injured users are classified into three categories of victims plus the uninjured

    1 - Unscathed

    2 - Killed

    3 - Hospitalized wounded

    4 - Light injury

sex: Sex of the user

    1 - Male

    2 - Female

Year_on: Year of birth of the user

trip: Reason for traveling at the time of the accident:

    1 - Home - work

    2 - Home - school

    3 - Shopping - Shopping

    4 - Professional use

    5 - Promenade - leisure

    9 - Other

secu: on 2 characters:
the first concerns the existence of a safety equipment

    1 - Belt

    2 - Helmet

    3 - Children's device

    4 - Reflective equipment

    9 - Other

the second is the use of Safety Equipment

    1 - Yes

    2 - No

    3 - Not determinable

locp: Location of the pedestrian:

On pavement:

    1 - A + 50 m from the pedestrian crossing

    2 - A - 50 m from the pedestrian crossing

On pedestrian crossing:

    3 - Without light signaling

    4 - With light signaling

Various:

    5 - On the sidewalk

    6 - On the verge

    7 - On refuge or BAU

    8 - On against aisle

actp: Action of the pedestrian:

Moving

    0 - not specified or not applicable

    1 - Meaning bumping vehicle

    2 - Opposite direction of the vehicle
    Various

    3 - Crossing

    4 - Masked

    5 - Playing - running

    6 - With animal

    9 - Other

etatp: This variable is used to specify whether the injured pedestrian was alone or not

    1 - Only

    2 - Accompanied

    3 - In a group

VEHICLES:

Num_Acc
Accident ID

Num_Veh
Identification of the vehicle taken back for each user occupying this vehicle (including pedestrians who are
attached to vehicles that hit them) - alphanumeric code

GP
Flow direction :

    1 - PK or PR or increasing postal address number

    2 - PK or PR or descending postal address number

CATV
Category of vehicle:

    01 - Bicycle

    02 - Moped <50cm3

    03 - Cart (Quadricycle with bodied motor) (formerly "cart or motor tricycle")

    04 - Not used since 2006 (registered scooter)

    05 - Not used since 2006 (motorcycle)

    06 - Not used since 2006 (side-car)

    07 - VL only

    08 - Not used category (VL + caravan)

    09 - Not used category (VL + trailer)

    10 - VU only 1,5T <= GVW <= 3,5T with or without trailer (formerly VU only 1,5T <= GVW <= 3,5T)

    11 - Most used since 2006 (VU (10) + caravan)

    12 - Most used since 2006 (VU (10) + trailer)

    13 - PL only 3,5T


### Characteristics Data

In [3]:
#AP: To get encoding of the csv since it's not the default (because French source of data)
with open(r'Datasets_cap3\characteristics.csv') as f:
    print(f)

<_io.TextIOWrapper name='Datasets_cap3\\characteristics.csv' mode='r' encoding='cp1252'>


In [4]:
df_char = pd.read_csv(r'Datasets_cap3\characteristics.csv', skip_blank_lines=True, encoding='cp1252', encoding_errors='ignore', low_memory=False)

In [5]:
#AP: Should have 839,985 rows, based on looking at csv
print(len(df_char))

839985


### holidays Data

In [6]:
#AP: To get encoding of the csv since it's not the default (because French source of data)
with open(r'Datasets_cap3\holidays.csv') as f:
    print(f)

<_io.TextIOWrapper name='Datasets_cap3\\holidays.csv' mode='r' encoding='cp1252'>


In [7]:
df_holiday = pd.read_csv(r'Datasets_cap3\holidays.csv', skip_blank_lines=True, encoding='cp1252', encoding_errors='ignore', low_memory=False)

In [8]:
#AP: Should have 132 rows, based on looking at csv
print(len(df_holiday))

132


### places Data

In [9]:
#AP: To get encoding of the csv since it's not the default (because French source of data)
with open(r'Datasets_cap3\places.csv') as f:
    print(f)

<_io.TextIOWrapper name='Datasets_cap3\\places.csv' mode='r' encoding='cp1252'>


In [10]:
df_places = pd.read_csv(r'Datasets_cap3\places.csv', skip_blank_lines=True, encoding='cp1252', encoding_errors='ignore', low_memory=False)

In [11]:
#AP: Should have 839,985 rows, based on looking at csv
print(len(df_places))

839985


### users Data

Had to rename file to 'crash_users.csv' from 'users.csv'because of unicode escape character error

In [ ]:
#AP: To get encoding of the csv since it's not the default (because French source of data)
with open(r'Datasets_cap3\crash_users.csv') as f:
    print(f)

<_io.TextIOWrapper name='Datasets_cap3\\crash_users.csv' mode='r' encoding='cp1252'>


In [ ]:
df_users = pd.read_csv(r'Datasets_cap3\crash_users.csv', skip_blank_lines=True, encoding='cp1252', encoding_errors='ignore', low_memory=False)

In [ ]:
#AP: Should have about 1.88 million rows, based on kaggle stats
print(len(df_users))

### vehicles Data

Had to rename file to 'crash_vehicles.csv' from 'ehicles.csv'because of unicode character error

AP: **Not using this dataset**. Research into each of the features suggests to me that they would not be beneficial in characterizing crash locations for purposes of predicting where they might occur.

If investigation into this data is needed, convert the cells back to code cells (currently as Raw so the code doesn't execute)

# Data Wrangling

**Data Wrangling Steps:** (Unordered list)

- Naming conventions (cols & values)
- Outliers
- nulls
    - Check for null replacements (i.e. "99", "0", etc.)
- duplicates
- Value consistency
- 

DF var names:
- df_char
    - (characteristics)
- df_holiday
    - (holidays)
- df_places
    - (places)
- df_users
    - (users)
- df_vehicles
    - (vehicles)

## Initial Exploration

In [ ]:
df_char.head()

In [ ]:
df_char.info()

In [ ]:
df_holiday.head()

In [ ]:
df_holiday.info()

In [ ]:
df_places.head()

In [ ]:
df_places.info()

In [ ]:
df_users.head()

In [ ]:
df_users.info()

## Naming conventions
cols & values

For clarity & some translation from French

Making new col names with orginal appended at end (for my own ease when referring to dataset documentation)

### df_char

In [ ]:
df_char.head(2)

In [ ]:
df_char.rename({'Num_Acc' : 'acc_id',
                'an' : 'year_an',
                'mois' : 'month_mois',
                'jour' : 'day_jour',
                'hrmn' : 'hrmn',
                'lum' : 'luminosity_lum',
                'agg' : 'built_up_agg',
                'int' : 'intersection_type_int',
                'atm' : 'weather_atm',
                'col' : 'collision_type_col',
                'adr' : 'address_drop',
                'gps' : 'gps_drop',
                'lat' : 'latitude_drop',
                'long' : 'longitude_drop',
                'dep' : 'dep_drop',
                'com' : 'commune_num_drop'},
               axis=1, inplace=True)
df_char.head(2)

### df_holiday

In [ ]:
df_holiday.head(2)

In [ ]:
list(df_holiday.holiday.unique())

#AP: Note that these are French holidays, keeping since this dataset is about French crashes

In [ ]:
df_holiday.rename({'ds' : 'date'}, axis=1, inplace=True)
print(list(df_holiday.columns))

### df_places

In [ ]:
df_places.sample(2)

In [ ]:
df_places.env1.value_counts()

In [ ]:
df_places.rename({'Num_Acc' : 'acc_id',
                  'catr' : 'road_category_catr',
                  'voie' : 'road_num_drop',
                  'v1' : 'v1_drop',
                  'v2' : 'v2_drop',
                  'circ' : 'road_type_circ',
                  'nbv' : 'lane_count_nbv',
                  'vosp' : 'reserved_lane_type_vosp',
                  'prof' : 'road_slope_prof',
                  'pr' : 'pr_drop',
                  'pr1' : 'pr1_drop',
                  'plan' : 'road_curvature_plan',
                  'lartpc' : 'central_sep_width_lartpc',
                  'larrout' : 'road_width_larrout',
                  'surf' : 'surface_cond_surf',
                  'infra' : 'infrastructure_infra',
                  'situ' : 'crash_location_situ',
                  'env1' : 'env1_drop'}, axis=1, inplace=True)
print(list(df_places.columns))

In [ ]:
df_places.head(2)

dropping env1 because values don't make sense and no var of that name in documentation from French Gov.t

### df_users

In [ ]:
df_users.head(2)

In [ ]:
df_users.rename({'Num_Acc' : 'acc_id',
                 'num_veh' : 'num_veh_drop',
                 'place' : 'place_in_veh_place',
                 'catu' : 'person_cat_catu',
                 'grav' : 'injury_severity_grav',
                 'sexe' : 'sexe_drop',
                 'trajet' : 'travel_reason_trajet',
                 'secu' : 'secu_drop',
                 'locp' : 'ped_location_locp',
                 'actp' : 'ped_action_actp',
                 'etatp' : 'ped_group_etatp',
                 'an_nais' : 'an_nais_drop'}, axis=1, inplace=True)
print(list(df_users.columns))

In [ ]:
df_users.head(2)

### df_vehicles
This data won't be used in this project.

Keeping code for possible back reference

## Drop unneeded cols

Dropping cols that I am reasonably sure **would not benefit characterization of accident locations or date/time analysis** for the use of preemptively deploying resources/personnel

ex. year of birth may have an effect on probability of a crash but agencies are unable to pre-screen drivers before they get on the road or determine the exact route the driver will take

In [ ]:
df_char.drop(['commune_num_drop', 'address_drop', 'gps_drop', 'latitude_drop', 'longitude_drop', 'dep_drop'], axis=1, inplace=True, errors= 'ignore')
print(list(df_char.columns))

In [ ]:
df_places.drop(['road_num_drop', 'v1_drop', 'v2_drop', 'pr_drop', 'pr1_drop', 'env1_drop'], axis=1, inplace=True, errors= 'ignore')
print(list(df_places.columns))

In [ ]:
df_users.drop(['sexe_drop', 'secu_drop', 'an_nais_drop', 'num_veh_drop'], axis=1, inplace=True, errors= 'ignore')
print(list(df_users.columns))

In [ ]:
print('Percent of total rows where ped location unspecified or not applicable')
print(round(df_users.ped_location_locp.value_counts(dropna=False).sort_index()[0] / len(df_users), 2) * 100, '%')

With about 92% of df_user rows with value of 0 (unspecified ped location), this col is not helpful for location characterization

**dropping col**

In [ ]:
df_users.ped_group_etatp.value_counts(dropna=False).sort_index()

In [ ]:
print(round(df_users.ped_group_etatp.value_counts(dropna=False).sort_index()[0] / len(df_users), 2) * 100, '%')

With about 92% of df_user rows with value of 0 (unspecified ped location), this col is not helpful for location characterization

**dropping col**

In [ ]:
df_users.drop(axis=1, inplace=True, errors= 'ignore', labels=['place_in_veh_place', 'travel_reason_trajet', 'ped_location_locp', 'ped_action_actp', 'ped_group_etatp'])
list(df_users.columns)

## Merge df's where possible
Making it easier to manage the data

In [ ]:
print('Row count of each df:')
for df in [df_char, df_holiday, df_places, df_users,
           # df_vehicles
          ]:
    print(len(df))

#AP: df_char & df_places have same row count, look into if they actually match

#AP: Explore 'acc_id' col to make sure it's actually unique per accident

In [ ]:
match_acc_id = df_char[df_char.acc_id == df_places.acc_id].acc_id
no_match_acc_id = df_char[df_char.acc_id != df_places.acc_id].acc_id

In [ ]:
print(len(match_acc_id))
print(len(no_match_acc_id))

#AP: accident ID's seem to match. Joining the tables.

In [ ]:
df_char_place = df_char.merge(df_places, on='acc_id', how='inner')

In [ ]:
print('df_char:\n', list(df_char.columns))
print()
print('df_places:\n', list(df_places.columns))
print()
print(list(df_char_place.columns))
print()
print('row count:', len(df_char_place))

### Explore df_vehicles to try & merge
It has a acc_id col but more rows than df_char_place. Want to see if I can reduce row count to match.

**This project doesn't need specific vehicle info**

The count of unique acc_id's seem to match. Now to see if plausible to aggregate so row counts match

Dropping vehicle_cat_catv col since multiple labels unused since 2006 present (in this 2005-2016 dataset)

AP: **Researching deeper into each of the features in the vehicles df, they do not seem that they would be helpful in characterizing where accidents occur for the purposes of predicting where they would occur.**

Converting df_vehicles cells to raw so the code doesn't run and errors are avoided

### Explore df_users to try & merge

There are multiple people per crash, trying to see if there is a way to group by acc_id in a way that doesn't loose quality of data.

In [ ]:
df_users.head(10)

In [ ]:
len(df_users)

In [ ]:
df_users.acc_id.nunique(dropna=False)

____
person_cat_catu col

In [ ]:
df_users.person_cat_catu.value_counts(dropna=False).sort_index()

AP: Note: There should only be values 1, 2, or 3 for this col. There are 3267 rows with value of 4.

**AP Note: Possible to create col to denote whether pedestrian present or not (0 : No or 1 : Yes). Presence of persons in vehicle is assumed since dataset is about vehicle crashes.**

Also possible to create col for count of persons involved. This could help agencies decide how much resources/personnel to deploy to the area in relation to how many people they expect to treat. This is currently outside the scope of the project.

____
injury_severity_grav col

In [ ]:
df_users.injury_severity_grav.value_counts(dropna=False).sort_index()

AP: Possible to create col that ranks severity of the accident based on total injuries. Would likely subtract 1 from each value in col (since 1 represents unharmed, converting it to zero would keep the final score more easily understood). A higher score would mean more severe accident in terms of people harmed, requiring more resources from the agency to deal with

This could also potentially be used as a tie breaker between locations with equal accident probabilities

#### Create cols for aggregation

In [ ]:
df_users.head()

In [ ]:
df_users.info()

In [ ]:
#AP: Double check for any nulls
df_users.isnull().sum()

____
from person_cat_catu, create col to denote 0 (No) or 1 (Yes) for pedestrian

In [ ]:
df_users.person_cat_catu.value_counts().sort_index()

Value of 3 represents pedestrian. Want to replace 3 with 1 & all other values with 0

In [ ]:
df_users['ped_present'] = df_users.person_cat_catu.replace({1:0, 2:0, 3:1, 4:0})

In [ ]:
#AP: Check whether I Created col correctly
print(df_users[df_users['person_cat_catu']==3].sample(5))
print()
print(df_users[df_users['person_cat_catu'].isin([1,2,4])].sample(5))

Now when I group by the acc_id col, I can just sum up the pedestrians (ped_present) and use that as needed in future analysis.

____
~from injury_severity_grav, create col to denote accident severity based on injuries~

~First to create col with each value reduced by 1.~

~Durring grouping/aggregation, can add the adjusted values together to get an accident severity score.~

### Create grouped/aggregated df for merging into the main df

In [ ]:
df_users_agg = df_users[['acc_id', 'ped_present'#,
                         #'injury_severity_adj_grav'
                        ]].groupby(by='acc_id', sort=False, dropna=False, as_index=False).sum()

In [ ]:
df_users_agg.head(15)

In [ ]:
df_users.head(15)

In [ ]:
df_users_agg.ped_present.value_counts(dropna=False).sort_index()

In [ ]:
len(df_users_agg)

~Change col name of ped_present to ped_count for clarity~

~Change col name of injury_severity_adj_grav to injury_count for clarity~

AP: Everything seems in place for merging df_users_agg into the main df

### Merge users data into main df

In [ ]:
df_main = df_char_place.merge(df_users_agg, on='acc_id', how='inner')

In [ ]:
print('df_char_place row count:', len(df_char_place))
print()
print('df_users_agg row count:', len(df_users_agg))
print()
print('df_main row count:', len(df_main))

In [ ]:
df_main.head(10)

In [ ]:
len(df_main)

## Create df_main cols for holiday & yes/no if holiday

Both to analyze if holidays have more accidents and also analyze which holidays have increased accidents.

In [ ]:
df_holiday.head(12) #because 11 holidays

In [ ]:
df_holiday.info()

In [ ]:
# Convert to datetime
df_holiday['date'] = pd.to_datetime(df_holiday['date'])

In [ ]:
df_holiday.info()

In [ ]:
df_holiday.holiday.value_counts()

____
## Create a 'date' col in datetime dtype

In [ ]:
df_main['hour'] = df_main['hrmn'] // 100
df_main['minute'] = df_main['hrmn'] % 100

In [ ]:
df_main[['hrmn', 'hour', 'minute']].head()

In [ ]:
df_main['date'] = pd.to_datetime(df_main[['year_an', 'month_mois', 'day_jour', 'hour', 'minute']]
                                 .rename(columns={'year_an': 'year', 'month_mois': 'month', 'day_jour': 'day'})
                                 .assign(year=lambda x: x['year']+2000))

In [ ]:
df_main[['year_an',
 'month_mois',
 'day_jour','hour',
 'minute','date']].head()

In [ ]:
df_main.dtypes

#### Round to the nearest hour

- Reasoning:
    - moving personnel & resources to a location requires enough time and effort that only having them manage that location for minutes is not worth it
    - Car accidents are almost always never coordinated to occur within minute ranges

In [ ]:
df_main['date'] = df_main['date'].dt.round('h')

In [ ]:
df_main[['date']].head()

#### Adjust 'hour' col to the new rounded hours

In order to use later for analysis on time of day

In [ ]:
df_main['hour'] = df_main['date'].dt.hour

In [ ]:
df_main['hour'].head()

#### Drop the cols year_an, month_mois, day_jour, & hrmn

Unneeded now

In [ ]:
df_main.drop(['year_an', 'month_mois', 'day_jour', 'hrmn', 'minute'], errors='ignore', axis=1, inplace=True)
print(df_main.columns)

### Create day_of_week col

This will later get converted with pd.get_dummies along with other categorical cols

In [ ]:
df_main['day_of_week'] = df_main['date'].dt.day_name()

In [ ]:
df_main[['date','day_of_week']].head(10)

### Create bool weekend
~& weekday cols~ Would just be inverses of each other, keeping is_weekend col

In [ ]:
df_main["is_weekend"] = df_main['date'].dt.day_name().isin(['Saturday', 'Sunday'])

In [ ]:
df_main[['date','day_of_week','is_weekend'
         #"is_weekday"
        ]].head(10)

____
### Create a col 'is_holiday'
of 0/1 (No, Yes) for whether or not it's a holiday.

In [ ]:
df_main['is_holiday'] = df_main['date'].isin(df_holiday['date'])

In [ ]:
df_main[['date', 'is_holiday']].sample(10)

In [ ]:
len(df_main)

____
### Create a col 'holiday' for noting which holiday is is.

AP: **Holding off on this nice-to-have for now.**

Can't currently figure out why an extra couple hundred rows are created when merging df_holidays & df_main.


For non-holidays, input value 'common_day'

## Test - Duplicate Values

In [ ]:
df_main[df_main.duplicated(keep=False)].sort_values(by='date')

## Check for nulls

In [ ]:
df_main.isnull().sum().sort_values(ascending=False)

____
Count total rows with any null

In [ ]:
null_row_count = df_main.isnull().any(axis=1).sum()
total_row_count = len(df_main)

print('null_row_count:', null_row_count)
print()
print('total_row_count:', total_row_count)
print()
print('Percent of rows with nulls:', round((null_row_count / total_row_count)*100, 2), '%')

**I am comfortable with dropping these rows since they account for < 1.32% of total rows in the df**

### Drop rows with any nulls

In [ ]:
df_main.dropna(ignore_index=True, inplace=True)

In [ ]:
df_main.isnull().sum().sort_values(ascending=False)

In [ ]:
null_row_count_2 = df_main.isnull().any(axis=1).sum()
total_row_count_2 = len(df_main)

print('null_row_count:', null_row_count_2)
print()
print('total_row_count:', total_row_count_2)
print()
print('Percent of rows with nulls:', round((null_row_count_2 / total_row_count_2)*100, 2), '%')

## Check dtypes
Convert if/when needed.

Need to input the strings for the categorical's since some can get fairly specific (ex. intersection type)

Planned to later use pd.get_dummies for the categorical features

In [ ]:
df_main.dtypes

AP: cols to convert:

- Categorical cols : create dicts in order to map numerical vals to the strings they represent. Then use get_dummies.
    - The train of thought here is making it easier to determine the specific characteristics of the crash locations rather than get a collection of numbers and try to translate them into words (ex. value of 3 for intersection type translates to T-intersection)
- float64 cols : convert to int64 (no col uses decimals)

In [ ]:
df_main.head()

### Categorical Cols

#### replace the strings for the categories in the categorical cols

In [ ]:
df_main.replace(inplace=True, to_replace = {'luminosity_lum' : {1 : 'lum_full_day', 2 : 'lum_twighlight', 3 : 'lum_night_no_light', 4 : 'lum_night_no_light', 5 : 'lum_night_yes_light'},
    'built_up_agg' : {1 : False, 2 : True},
    'intersection_type_int' : {0 : 'other', 1 : 'out_of_intersection', 2 : 'x_intersection', 3 : 't_intersection', 4 : 'y_intersection', 5 : '4+_intersection', 6 : 'roundabout', 7 : 'place', 8 : 'level_crossing', 9 : 'other'},
    'weather_atm' : {1 : 'normal', 2 : 'rain_light', 3 : 'rain_heavy', 4 : 'snow_hail', 5 : 'fog_smoke', 6 : 'storm', 7 : 'dazzling', 8 : 'cloudy', 9 : 'other'},
    'collision_type_col' : {1 : 'two_cars_frontal', 2 : 'two_cars_rear', 3 : 'two_cars_side', 4 : 'three+_chain', 5 : 'three+_multiple', 6 : 'other', 7 : 'no_collision'},
    'road_category_catr' : {1 : 'autoroutes ', 2 : 'national_road', 3 : 'departmental_road', 4 : 'municipal_road', 5 : 'off_public_network', 6 : 'parking_lot', 9 : 'other'},
    'road_type_circ' : {0.0 : 'other', 1 : 'one_way', 2 : 'two_way', 3 : 'seperated_carriageway', 4 : 'variable_assignment'},
    'reserved_lane_type_vosp' : {0.0 : 'no_reserved_lane', 1 : 'bike_path', 2 : 'bike_parking', 3 : 'reserved_lane'},
    'road_slope_prof' : {0.0 : 'other', 1 : 'dish', 2 : 'slope', 3 : 'hill_top', 4 : 'hill_bottom'},
    'road_curvature_plan' : {0.0 : 'other', 1 : 'straight', 2 : 'curve_left', 3 : 'curve_right', 4 : 's_shape'},
    'surface_cond_surf' : {0.0 : 'other', 1 : 'dry', 2 : 'wet', 3 : 'puddles', 4 : 'flooded', 5 : 'snow', 6 : 'mud', 7 : 'ice', 8 : 'oil', 9 : 'other'},
    'infrastructure_infra' : {0.0 : 'none', 1 : 'tunnel', 2 : 'bridge', 3 : 'interchange', 4 : 'railway', 5 : 'carrefour_arranged', 6 : 'pedestrian_area', 7 : 'toll_zone'},
    'crash_location_situ' : {0.0 : 'none', 1 : 'road', 2 : 'emergency_stop_lane', 3 : 'verge', 4 : 'sidewalk', 5 : 'bike_path'}})

In [ ]:
df_main.sample(5)

##### check for any missed values
collapsed since large output

In [ ]:
#check for any missed values
cat_cols = ['luminosity_lum', 'built_up_agg', 'intersection_type_int', 'weather_atm', 'collision_type_col',
            'road_category_catr', 'road_type_circ', 'reserved_lane_type_vosp', 'road_slope_prof', 'road_curvature_plan',
            'surface_cond_surf', 'infrastructure_infra', 'crash_location_situ', 'day_of_week']
for col in cat_cols:
    print(df_main[col].value_counts(dropna=False))
    print()

#### convert to 'category' type

In [ ]:
df_main[cat_cols] = df_main[cat_cols].astype('category')

In [ ]:
df_main.dtypes

____
### Numerical Cols

In [ ]:
num_cols = ['lane_count_nbv', 'central_sep_width_lartpc', 'road_width_larrout']
#Not including any engineered features

In [ ]:
df_main[num_cols].dtypes

In [ ]:
#Check for any values that actually use decimals (meaning the col shouldn't be turned into 'int' type)
for col in num_cols:
    df_floats = df_main[[col]] % 1 != 0
    print(df_floats[[col]].sum())
    print()

Convert to 'int' type

In [ ]:
df_main[num_cols] = df_main[num_cols].astype('int')
df_main[num_cols].dtypes

~### Convert ped_present to bool~

I learned that int type with 0 & 1 is easier for models in general than bool True/False

## Check num_cols for value consistency & expected ranges
cat_cols already ensured & checked when I manually replaced the strings for them above.

In [ ]:
df_main[num_cols].info()

In [ ]:
df_main[num_cols].nunique()

AP: that's a suspsiciously high count of unique values for number of road lanes

The width cols also seem to have an unexpectedly high count of unique values. It is France and I could just be unfamiliar with their road standards.

### lane_count_nbv check

In [ ]:
#AP: df.value_counts() since 53 seems manageable for viewing

#df_main['lane_count_nbv'].value_counts().sort_values(ascending=False)

df_main['lane_count_nbv'].value_counts().sort_index()

AP: There **shouldn't be zero lanes for a road** AND the **most lanes I could find for French roads is 8** (from wikipedia & thouroughly exploring google maps)

Zero lanes could be for non-road crashes, needs investigation

In [ ]:
len(df_main)

In [ ]:
#Count of rows with lanes =0 or >8
#and percent of total rows
print('Count of rows with lanes =0 or >8: ', ((df_main['lane_count_nbv'] == 0) | (df_main['lane_count_nbv'] > 8)).sum())
print('Percent of total rows: ', round((((df_main['lane_count_nbv'] == 0) | (df_main['lane_count_nbv'] > 8)).sum() / len(df_main)) * 100, 2), '%')

In [ ]:
# count of rows with zero lanes (it's listed in value_counts but I like to see it isolated)
print('count rows of 0 lanes: ', (df_main['lane_count_nbv'] == 0).sum())
print('Percent: ', round(((df_main['lane_count_nbv'] == 0).sum() / len(df_main)) * 100, 2), '%')

In [ ]:
# count of rows above 8 lanes
print('rows with >8 lanes: ', (df_main['lane_count_nbv'] > 8).sum())
print('Percent: ', round(((df_main['lane_count_nbv'] > 8).sum() / len(df_main)) * 100, 2), '%')

Is there a specific road type that maybe I can attribute high & 0 lanes too?

In [ ]:
df_main.plot(kind='scatter', x='road_category_catr', y='lane_count_nbv')
plt.xticks(rotation=10)
plt.axhline(y=0, color='r', linestyle='--')
plt.axhline(y=8, color='r', linestyle='--')
plt.show()

#AP: added horizontal lines for y=0 (min lanes) & y=8 (max lanes I'll allow)

No specific road type I can inveestigate for high or 0 lane count.

In [ ]:
sns.scatterplot(data=df_main, x=df_main['date'].dt.year, y=df_main['lane_count_nbv'])
plt.axhline(y=0, color='r', linestyle='--')
plt.axhline(y=8, color='r', linestyle='--')
plt.show()

~2008 & ealier have the high values. Check to see the range of values for 2009 & later in order to figure out what I could replace the high values with~

Will be dropping rows with >8 lanes, accounts for <0.5% of total data

**AP: Seems that there's a 2017 year, probably just have to drop that single row**
#### Drop 2017 row

In [ ]:
df_main[df_main['date'].dt.year == 2017].index

In [ ]:
df_main.drop(df_main[df_main['date'].dt.year == 2017].index, inplace=True)

In [ ]:
df_main[df_main['date'].dt.year == 2017].index

~AP: Starting at 2009, I want to see what the max value for lane count is~

#### Drop rows with >8 lanes

converting prior used cells to raw so they don't eecute and I can keep the code for later reference.|

~What kind of locations are these high number lanes?~

In [ ]:
df_main.drop(df_main[df_main['lane_count_nbv'] > 8].index, axis=0, inplace=True)

In [ ]:
df_main['lane_count_nbv'].value_counts(dropna=False).sort_index()

AP: It is unclear (even from official documentation) whether lanes include ramps for highways. I am deciding to leave this as is.

#### Check zero lane roads

In [ ]:
df_main[df_main['lane_count_nbv'] == 0].sample(5)

Plot value counts for cols corresponding to road type characteristics for lanes = 0

In [ ]:
for col in ['built_up_agg', 'intersection_type_int', 'road_category_catr', 'road_type_circ', 'reserved_lane_type_vosp', 'central_sep_width_lartpc', 'road_width_larrout', 'infrastructure_infra']:
    print(df_main[df_main['lane_count_nbv'] == 0][col].value_counts().sort_values(ascending=False))
    print()

In [ ]:
#warnings.filterwarnings('ignore')

columns_to_plot = ['built_up_agg', 'intersection_type_int', 'road_category_catr', 'road_type_circ', 'reserved_lane_type_vosp', 'central_sep_width_lartpc', 'road_width_larrout', 'infrastructure_infra', 'crash_location_situ']
n_cols = len(columns_to_plot)
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(10, 10), layout="constrained")
axes = axes.flatten()

for i, col in enumerate(columns_to_plot):
    sns.histplot(df_main[col], ax=axes[i])
    axes[i].set_title(col)
    #axes[i].set_xlabel('')  # Remove x-axis labels for clarity if needed
    #xes[i].set_ylabel('')
    axes[i].tick_params(axis='x', labelrotation=15, labelsize=7)
#plt.tight_layout()
plt.show()

#warnings.resetwarnings()

Majority characteristics for 0-lane roads:
- In built up areas
    - (built_up_agg = True)
- Out of intersection
    - intersection_type_int = out_of_intersection
    - Implies either off a road or on stretch of road without intersection
- municipal road type & departmental road
    - road_category_catr = municipal_road OR departmental_road
    - essentially a county road
    - departmental road is essentially a toll-free highway
- Two way road
    - road_type_circ = two_way
- No reserved lane
    - reserved_lane_type_vosp = no_reserved_lane
- no central separator in road
    - central_sep_width_lartpc = 0
- Majority of roads have a width of 0
    - road_width_larrout = 0
    - Strongly suggests this feature as the next to investigate for consistent and expected values
- No infrastructure
    - infrastructure_infra = none

**Plan for replacing 0's :**

base it off of road_desc_cols (that I'll create below)

In [ ]:
df_main[df_main['lane_count_nbv']==0]['lane_count_nbv'].value_counts(dropna=False).sort_index()

In [ ]:
road_desc_cols = ['built_up_agg', 'intersection_type_int', 'road_category_catr', 'road_type_circ', 'reserved_lane_type_vosp', 'infrastructure_infra', 'crash_location_situ']

In [ ]:
warnings.filterwarnings('ignore')

for col in road_desc_cols:
    sns.histplot(data=df_main[df_main['lane_count_nbv']==0][[col]], x=df_main[df_main['lane_count_nbv']==0][col])
    plt.title(col)
    plt.xticks(rotation=15)
    plt.figure(figsize=(6,3))
    plt.show

warnings.resetwarnings()

In [ ]:
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['road_type_circ'] == 'one_way') & 
    (df_main['road_category_catr'] == 'municipal_road'),
    'lane_count_nbv'
] = 1

In [ ]:
df_main[df_main['lane_count_nbv']==0]['lane_count_nbv'].value_counts(dropna=False).sort_index()

In [ ]:
df_main[df_main['lane_count_nbv']==0]['crash_location_situ'].value_counts()

In [ ]:
#duplicates check

df_main[df_main.duplicated(keep=False)].sort_values(by='date')

In [ ]:
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    ((df_main['road_category_catr'] == 'autoroutes') |
        (~df_main['road_category_catr'].isin(['other', 'parking_lot'])) |
        (df_main['road_type_circ']!='other') |
        (df_main['intersection_type_int']!='other') |
        (df_main['reserved_lane_type_vosp']!='no_reserved_lane') |
        (df_main['central_sep_width_lartpc'] != 0) |
        (df_main['road_slope_prof'] != 'other') |
        (df_main['road_curvature_plan'] != 'other') |
        (~df_main['infrastructure_infra'].isin(['none', 'pedestrian_area', 'railway'])) |
        (~df_main['crash_location_situ'].isin(['none', 'bike_path']))
    ),'lane_count_nbv'
    ] = 2

In [ ]:
#Ensure 0 lanes for applicable situations (i.e. parking lots)
#make sure not to add 0 lane observations. Noticed that 
df_main.loc[(df_main['lane_count_nbv'] != 0) & 
    (df_main['road_category_catr'] == 'parking_lot'),
    'lane_count_nbv'
    ] = 0

In [ ]:
df_main[df_main['lane_count_nbv']==0]['lane_count_nbv'].value_counts(dropna=False).sort_index()

Plot distributions of vals for 0 lanes:

In [ ]:
warnings.filterwarnings('ignore')

for col in road_desc_cols:
    sns.histplot(data=df_main[df_main['lane_count_nbv']==0][[col]], x=df_main[df_main['lane_count_nbv']==0][col])
    plt.title(col)
    plt.xticks(rotation=25)
    plt.figure(figsize=(6,3))
    plt.show
    
warnings.resetwarnings()

Still some rows where lanes shouldn't be zero

In [ ]:
for col in road_desc_cols:
    print(df_main[df_main['lane_count_nbv']==0][col].value_counts(dropna=False))
    print()

Where should not be 0 lanes: (& how many lanes can be reasonably & minimally expected)

- intersection_type_int
    - x_intersection
        - 2
    - t_intersection
        - 2
    - roundabout
        - 1
    - 4+_intersection
        - 2
    - y_intersection
        - 2
    - level_crossing
        - 1
- road_type_circ
    - two_way
        - 2
    - one_way
        - 1
    - variable_assignment
        - 1
    - seperated_carriageway
        - 2
- reserved_lane_type_vosp
    - reserved_lane
        - 1
- infrastructure_infra
    - tunnel
        - 1
    - carrefour_arranged
        - 2
    - bridge
        - 1
    - interchange
        - 2
    - toll_zone
        - 1
- crash_location_situ
    - road
        - 2 (reason: most roads are two lane (two-way))
    - verge
        - 1
    - emergency_stop_lane
        - 2

In [ ]:
#changing one-by-one

#intersection_type_int 2 lanes
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['intersection_type_int'].isin(['x_intersection', 't_intersection', '4+_intersection', 'y_intersection']))
    , 'lane_count_nbv'] = 2

#intersection_type_int 1 lane
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['intersection_type_int'].isin(['roundabout', 'level_crossing']))
    , 'lane_count_nbv'] = 1

#road_type_circ 2 lanes
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['road_type_circ'].isin(['two_way', 'seperated_carriageway']))
    , 'lane_count_nbv'] = 2

#road_type_circ 1 lane
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['road_type_circ'].isin(['one_way', 'variable_assignment']))
    , 'lane_count_nbv'] = 1

#reserved_lane_type_vosp 1 lane
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['reserved_lane_type_vosp'].isin(['reserved_lane']))
    , 'lane_count_nbv'] = 1

#infrastructure_infra 2 lanes
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['infrastructure_infra'].isin(['carrefour_arrange', 'interchange']))
    , 'lane_count_nbv'] = 2

#infrastructure_infra 1 lane
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['infrastructure_infra'].isin(['tunnel', 'bridge', 'toll_zone']))
    , 'lane_count_nbv'] = 1

#crash_location_situ 2 lanes
df_main.loc[(df_main['lane_count_nbv'] == 0) &
    (df_main['crash_location_situ'].isin(['road', 'emergency_stop_lane']))
    , 'lane_count_nbv'] = 2

#crash_location_situ 1 lane
df_main.loc[(df_main['lane_count_nbv'] == 0) & 
    (df_main['crash_location_situ'].isin(['verge']))
    , 'lane_count_nbv'] = 2

In [ ]:
for col in road_desc_cols:
    print(df_main[df_main['lane_count_nbv']==0][col].value_counts(dropna=False))
    print()

That puts the lane crisis to bed
___

### central_sep_width_lartpc check

The presence of a separator should be enough rather that the actual width in meters.

**Plan on adding a ~bool col of True/False~ int type (0/1) for presence of separator, then dropping this col**

In [ ]:
print(df_main['central_sep_width_lartpc'].value_counts())
df_main['central_sep_width_lartpc'].hist()

#### Create ~bool~ int (0/1) col for sep_present

In [ ]:
df_main['sep_present'] = df_main['central_sep_width_lartpc'] == 0

In [ ]:
print(df_main[['sep_present']].dtypes)
df_main[['sep_present']].head(8)

In [ ]:
df_main['sep_present'] = df_main['sep_present'].astype('int')

In [ ]:
print(df_main[['sep_present']].dtypes)
df_main[['sep_present']].head(8)

#### Drop central_sep_width_lartpc col

In [ ]:
df_main.drop('central_sep_width_lartpc', axis=1, inplace=True)

In [ ]:
'central_sep_width_lartpc' in df_main.columns

In [ ]:
df_main['sep_present'].value_counts()

### road_width_larrout check

In [ ]:
print(df_main['road_width_larrout'].value_counts().sort_index())
print('\nnunique:',df_main['road_width_larrout'].nunique())
df_main['road_width_larrout'].hist()

In [ ]:
len(df_main[df_main['road_width_larrout'] > 0])

##### **Dropping col**

Dropping based on usefullness of knowing width of the road for purpose of knowing where to send personnel & resources. The width of the road is not normally easily known, especially ot readily available during emergency situtations.

In [ ]:
df_main.drop('road_width_larrout', axis=1, inplace=True)

In [ ]:
'road_width_larrout' in df_main.columns

## Drop more columns

- acc_id
- collision_type_col
    - would be useful if predicting severity of crash. Currently just interested in if specific location or conditions would have high crash probability
- crash_location_situ
    - post-crash info. would be potentially useful if predicting severity of crash.

In [ ]:
df_main.drop(columns=['acc_id', 'collision_type_col', 'crash_location_situ'], inplace=True)

In [ ]:
for col in ['acc_id', 'collision_type_col', 'crash_location_situ']:
    print(f'{col} col in df_main: {col in df_main.columns}')

In [ ]:
##reset index, mostly as a just-in-case. Looks fine from doing df.head(), just want to be sure
df_main.reset_index(drop=True, inplace=True)

In [ ]:
df_main.tail()

## convert to int type

Bools &

['hour', sep_present, & lane_count_nbv] from int32 to int64 just for consistency

In [ ]:
df_main.dtypes

In [ ]:
#'hour', 'sep_present', & 'lane_count_nbv' cols

df_main[['hour', 'sep_present', 'lane_count_nbv']] = df_main[['hour', 'sep_present', 'lane_count_nbv']].astype('int64')
print(df_main[['hour', 'sep_present', 'lane_count_nbv']].dtypes)

In [ ]:
#is_weekend & is_holiday cols

df_main[['is_weekend', 'is_holiday']] = df_main[['is_weekend', 'is_holiday']].astype('int64')
print(df_main[['is_weekend', 'is_holiday']].dtypes)

## A check before doing Full EDA

In [ ]:
df_main.head()

In [ ]:
df_main.info()

## Reorder cols to group by general theme
Easier for me to mentally parse.

Themes such as "road description", "driving conditions", "weather condition", "temporal description"

In [ ]:
df_main = df_main[[
    #road desc.
    'road_category_catr', 'road_type_circ', 'intersection_type_int', 'lane_count_nbv', 'reserved_lane_type_vosp', 'sep_present', 'built_up_agg', 'road_slope_prof', 'road_curvature_plan', 'infrastructure_infra',
    #driving cond.
    'surface_cond_surf', 'ped_present',
    #weather
    'weather_atm', 'luminosity_lum',
    #Temporal
    'date', 'hour', 'day_of_week', 'is_weekend', 'is_holiday']]

#make lists of col names for each group (less typing for future me)
desc_road = ['road_category_catr', 'road_type_circ', 'intersection_type_int', 'lane_count_nbv', 'reserved_lane_type_vosp', 'sep_present', 'built_up_agg', 'road_slope_prof', 'road_curvature_plan', 'infrastructure_infra']
desc_driving = ['surface_cond_surf', 'ped_present']
desc_weather = ['weather_atm', 'luminosity_lum']
desc_time = ['date', 'hour', 'day_of_week', 'is_weekend', 'is_holiday']

In [ ]:
#make sure I didn't miss any
print(len(desc_driving + desc_road + desc_time + desc_weather))
print(len(df_main.columns))

In [ ]:
df_main.head(2)

# EDA

Hypotheses:
- majority of crashes occur for:
    - conditions:
        - no light
        - bad weather (heavy rain, storm)
        - common slippery surface conditions (ice, snowy, wet)
    - road types:
        - built-up
        - out_of_intersection
        - Municipal road type
        - two-way road
    - Time conditions
        - weekends & holidays

In [ ]:
df_main.head()

In [ ]:
df_main.info()

## Summary Stats

In [ ]:
#road description
df_main[desc_road].describe(include='all')

In [ ]:
#desc_driving
df_main[desc_driving].describe(include='all')

In [ ]:
#desc_weather
df_main[desc_weather].describe(include='all')

In [ ]:
#desc_time
df_main[desc_time].describe(include='all')

## Single Feature Distributions

road description

In [ ]:
warnings.filterwarnings('ignore')

#added percentage annotations
fig, axes = plt.subplots(5, 2, layout='tight', figsize=(10, 15))
axes = axes.flatten()

for ax, col in zip(axes, desc_road):
    if df_main[col].dtype == 'category' or df_main[col].dtype == 'bool':
        df_main[col].value_counts().plot(kind='bar', ax=ax, include_bool=True)

        total = df_main[col].count()
        for p in ax.patches:
            height = p.get_height()
            percent = 100 * height / total
            ax.annotate(f'{percent:.1f}%', 
                        (p.get_x() + p.get_width() / 2, height), 
                        ha='center', va='center', 
                        xytext=(0, 5), 
                        textcoords='offset points')

    else:
        df_main[col].plot(kind='hist', ax=ax)
            #note: sns.histplot was having 4 lanes as highest proportion, switched to pandas/matplotlib syntax
        counts, bins = np.histogram(df_main[col], bins='auto')
        total = counts.sum()
        bin_centers = 0.5 * (bins[:-1] + bins[1:])
        for center, count in zip(bin_centers, counts):
            percent = 100 * count / total
            ax.annotate(f'{percent:.1f}%', 
                        (center, count), 
                        ha='center', va='bottom', 
                        xytext=(0, 5), 
                        textcoords='offset points')

    ax.set_title(f'Distribution of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=15)
    ax.yaxis.grid(True)

plt.show()

warnings.resetwarnings()

**Observations:**

- municipal & departmental roads have majority of crashes
- two way roads
- out of intersection
    - can imply off road as well as parts of road between intersections
- 2 lane roads
- no reserved lane
    - follows that municipal roads don't usually have a reserved lane
- separator is commonly present (separating opposite traffic direction)
- Commonly in built up areas
    - ex. within a city rather than in the country-side
- dish road slope
    - i.e. a valley between 2 hills
- straight roads are where majority of crashes occur
    - Makes sense based on assumption that majority of road length is straight (would require deeper research to verify)
    - Previously thought that 's_shape' would have higher proportion
- no infrastructure present for most crashes

desc_driving

In [ ]:
warnings.filterwarnings('ignore')

#added percentage annotations
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained', figsize=(8, 4))

for ax, col in zip(axes, desc_driving):
    df_main[col].value_counts().plot(kind='bar', ax=ax, include_bool=True)

    total = df_main[col].count()
    for p in ax.patches:
        height = p.get_height()
        percent = 100 * height / total
        ax.annotate(f'{percent:.1f}%', 
                    (p.get_x() + p.get_width() / 2, height), 
                    ha='center', va='center', 
                    xytext=(0, 5), 
                    textcoords='offset points')

    ax.set_title(f'Distribution of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=20)
    ax.yaxis.grid(True)

plt.show()

warnings.resetwarnings()

**Observations:**

- Dry roads are most common with wet road surfaces being a distant second ('other' being even more distant third)
    - Makes sense as dry conditions are extremely common compared to any other
- Pedestrians are not commonly present in a crash

desc_weather

In [ ]:
warnings.filterwarnings('ignore')

#added percentage annotations
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained', figsize=(8, 4))

for ax, col in zip(axes, desc_weather):
    df_main[col].value_counts().plot(kind='bar', ax=ax, include_bool=True)

    total = df_main[col].count()
    for p in ax.patches:
        height = p.get_height()
        percent = 100 * height / total
        ax.annotate(f'{percent:.1f}%', 
                    (p.get_x() + p.get_width() / 2, height), 
                    ha='center', va='center', 
                    xytext=(0, 5), 
                    textcoords='offset points')

    ax.set_title(f'Distribution of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=20)
    ax.yaxis.grid(True)

plt.show()

warnings.resetwarnings()

**Observations:**

- Majority is 'normal' weather
    - implying reasonable sunshine for visibility, sparse cloud cover, and no percipitation
- light rain is distant second weather condition for crashes
- full daylight is present for majority of crashes
    - Indicates that crash occured during day time
        - (time-series plots below show most crashes occur between 8am-7pm) (sunset time varies through the year but avg's after 7pm (https://sunrise-sunset.org/france))
    - Makes sense as 'normal' weather is most common
- night time with artificial lighting is distantly second most common for crashes
    - Original expectation was that less lighting (lum_night_no_light or lum_twighlight) would be in greater proportion

desc_time

In [ ]:
warnings.filterwarnings('ignore')

#added percentage annotations
fig, axes = plt.subplots(nrows=3, ncols=2, layout='constrained', figsize=(12, 10))
axes = axes.flatten()

for ax, col in zip(axes, desc_time):
    if col == 'date':
        monthly_counts = df_main.set_index('date')
        monthly_counts = monthly_counts.resample('M').size()
        monthly_counts.plot(ax=ax)
        #df_main.set_index('date')[col].resample('M').size().plot(ax=ax)
        ax.set_title(f'Monthly Trend of Records')
        ax.set_xlabel('Date')
        ax.set_ylabel('Count')

    elif col == 'hour':
        sns.histplot(df_main[col], bins=24, ax=ax)

        counts, bins = np.histogram(df_main[col], bins=24)
        total = counts.sum()
        bin_centers = 0.5 * (bins[:-1] + bins[1:])
        for center, count in zip(bin_centers, counts):
            percent = 100 * count / total
            ax.annotate(f'{percent:.1f}%', 
                        (center, count), 
                        ha='center', va='bottom', 
                        xytext=(0, 5), 
                        textcoords='offset points')


    elif df_main[col].dtype == 'category' or df_main[col].dtype == 'bool':
        df_main[col].value_counts().plot(kind='bar', ax=ax, include_bool=True)

        total = df_main[col].count()
        for p in ax.patches:
            height = p.get_height()
            percent = 100 * height / total
            ax.annotate(f'{percent:.1f}%', 
                        (p.get_x() + p.get_width() / 2, height), 
                        ha='center', va='center', 
                        xytext=(0, 5), 
                        textcoords='offset points')

    else:
        df_main[col].plot(kind='hist', ax=ax)

        counts, bins = np.histogram(df_main[col], bins='auto')
        total = counts.sum()
        bin_centers = 0.5 * (bins[:-1] + bins[1:])
        for center, count in zip(bin_centers, counts):
            percent = 100 * count / total
            ax.annotate(f'{percent:.1f}%', 
                        (center, count), 
                        ha='center', va='bottom', 
                        xytext=(0, 5), 
                        textcoords='offset points')

    ax.set_title(f'Distribution of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=20)
    ax.yaxis.grid(True)

plt.show()

warnings.resetwarnings()

**Observations:**

- (Time-series check) Crashes seem to be declining in a linear fashion over the years
- Time frame for elevated crashes is 8am - 7pm
    - Peak at 6pm
    - (sunset time varies through the year but avg's after 7pm (https://sunrise-sunset.org/france))
- Friday has the most crashes
    - Sunday has least amount of crashes
    - rest of days are near even in proportion (monday is slightly less but not by a lot)
- Not usually a weekend for crashes
    - Makes sense as friday is most common crash day
    - And that weekdays are more common than weekends
- Super majority of crashes on non-holidays
    - Makes sense as holidays are not common (especially with only looking at 11 days out of the 365 day year)

## Bivariate Analysis

### Categorical-Categorical Feature Relationships

#### Chi-Square Test for Independence
**Outputing only p-values != 0** since it'd be quite a lot of output otherwise

If p-value = 0 then that denotes statistical significance of the difference of actual & expected, strongly suggesting a relationship between the two features

___
desc_road

In [ ]:
print(desc_road)

In [ ]:
for col1 in desc_road:
    for col2 in desc_road:
        if (df_main[col].dtype == 'category') | (df_main[col].dtype == 'bool'):
            if col1 != col2:
                contingency_table = pd.crosstab(df_main[col1], df_main[col2])
                chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
                if p != 0 :
                    print(f'Chi-Square Test for {col1} vs {col2}: Chi2 = {chi2:.2f}, p-value = {p}')

Output has p-values at essentially 0, much less than 0.05 for the commonly used significance level.

**Each feature in listed in desc_road has a statistically significant relationship to each other**

___
desc_driving

In [ ]:
print(desc_driving)

In [ ]:
for col1 in desc_driving:
    for col2 in desc_driving:
        if (df_main[col].dtype == 'category') | (df_main[col].dtype == 'bool'):
            if col1 != col2:
                contingency_table = pd.crosstab(df_main[col1], df_main[col2])
                chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
                if p != 0 :
                    print(f'Chi-Square Test for {col1} vs {col2}: Chi2 = {chi2:.2f}, p-value = {p}')

Output has p-values at essentially 0, much less than 0.05 for the commonly used significance level.

**Each feature in listed in desc_driving has a statistically significant relationship to each other**

___
desc_weather

In [ ]:
print(desc_weather)

In [ ]:
for col1 in desc_weather:
    for col2 in desc_weather:
        if (df_main[col].dtype == 'category') | (df_main[col].dtype == 'bool'):
            if col1 != col2:
                contingency_table = pd.crosstab(df_main[col1], df_main[col2])
                chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
                if p != 0 :
                    print(f'Chi-Square Test for {col1} vs {col2}: Chi2 = {chi2:.2f}, p-value = {p}')

No output with p-values != 0.

**Each feature in listed in desc_weather has a statistically significant relationship to each other**

___
desc_time

In [ ]:
print(desc_time)

In [ ]:
for col1 in desc_time:
    for col2 in desc_time:
        if (df_main[col].dtype == 'category') | (df_main[col].dtype == 'bool'):
            if col1 != col2:
                contingency_table = pd.crosstab(df_main[col1], df_main[col2])
                chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
                if p != 0 :
                    print(f'Chi-Square Test for {col1} vs {col2}: Chi2 = {chi2:.2f}, p-value = {p}')

No relationship:
- is_weekend & is_holiday

All other features have a relationship

### Categorical - Numerical Relationships

Essentially a check that 0 lane roads were handled correctly. Since lanes col is only actual numerical col (hour is int type but is essentially ordered categorical)

Converting cells to raw to not execute the code.

___
desc_road

In [ ]:
print(desc_road)

___
desc_driving

In [ ]:
print(desc_driving)

___
desc_weather

In [ ]:
print(desc_weather)

___
desc_time

In [ ]:
print(desc_time)

## Observations Summary

- Crashes seem to be declining in a linear fashion over the years

Road Characteristics:
- Municipal (local) roads account for about 50% of crashes
    - highways (departmental, national, autoroutes) account for about 47% of crashes
- ~64% of crashes occur on two-way roads
- ~72% of crashes occur outside any intersection
    - Proving a starting hypothesis wrong
- ~84% of crashes have a separator present (separating bi-directional roads)
- ~69% occur in built-up areas (ex. within towns or cities)
- ~76% occur on dish shaped stretches of road
    - low point between two higher points (ex. \_/)
- ~76% occur on straight strethces of road
- ~89% occur away from road infrastructure (such as toll zones or bridges)

Driving Conditions:
- ~78% occur with a dry road
- ~83% occur with no pedestrians present

Weather Conditions:
- ~81% of crashes occur during normal weather (light out, sparse clouds, no precipitation)
    - ~10% occur during rain
- ~69% occur during the day
    - ~17% occur at night with artificial lighting
    - Disproves starting hypothesis that more crashes occur with minimized light

Time Conditions:
- Hour of day range for elevated crashes:
    - 8am-7pm
    - peak at 6pm (10.4%)
- Friday is most common day of week at 16.7%
    - Other days (except Sunday) hover around 14%
    - Sunday about 12%
- Crashes usually occur on weekdays
- Almost no crashes occur on a holiday

Bivariate Relationships:
- All features related to each other within themed groups except:
    - weekends and holidays
- Makes sense as presence of road characteristics are usually dependent on others

# Next Steps

- Encode categorical features with pd.get_dummies in next notebook. (one-hot)
- Split off 'date' col when analyzing actual locations
    - Use it for doing time-series analysis
- Create col ('loc_num') to categorize each possible combo of all features
    - numbered 1 through n for each unique feature combo
- Use loc_num to synthetically create non-crash observations
    - Allows for model to be used for supervised classification of Yes/No for a crash
    - Need to label True/False for crash too
- Start off the modeling phase with logistic regression
    - Can get coeff's for each feature for how much it effects True/False for crash
- Make neural network model for predicting crash
- Time-series analysis (?)

# Export df as csv
convert back to code cell if needed

# Pre-Processing

Planned Steps: (tentative plan)

- Create dummy or indicator features for categorical variables
    - Plan on pd.get_dummies to one-hot encode
- Create 'crash' col
    - if crash then 1, if not crash then 2
    - As df is currently, every row gets crash=1
- Artificially create crash=0 observations
    - Will need to create a matrix of all the possible combos of features then assign crash=0 to those combos not found in the starting df
- ~Standardize the magnitude of numeric features~
    - No numeric/continuous features in data (all categorical in nature)
- Split into testing and training datasets

In [ ]:
print(df_main.info())
df_main.head()

## Drop date col
will be helpful for time analysis later, already extracted info from date/time that seems most helpful for characterizing crash locations

In [ ]:
df_main2 = df_main.drop(columns='date')

In [ ]:
'date' in df_main2.columns

## Convert all cols to category
(so pd.get_dummies works the way I intend)

In [ ]:
df_main2.info()

In [ ]:
df_main2[df_main2.select_dtypes('int64').columns] = df_main2[df_main2.select_dtypes('int64').columns].astype('category')


print(df_main2.info())

## Dummy creation

In [ ]:
df_dummies = pd.get_dummies(df_main2)

In [ ]:
print(df_dummies.info())
df_dummies.head()

## Create 'crash' col

    To denote whether crash occured or not.
    Every row will have crash=1
    plan on creating crash=0 rows based on feature combos not already present in the dataset

In [ ]:
df_dummies['crash'] = 1

In [ ]:
df_dummies.info()

In [ ]:
df_dummies.head()

# Artificial crash=0 observations
- create a matrix of all the possible feature combos
- for each combo that does not already appear in df_dummies, assign crash=0

## Generate all possible combinations of features

In [ ]:
feat_cols = df_dummies.columns[:-1]  # Exclude the target variable 'crash'
print('crash' in feat_cols)

### get feature combos already present

In [ ]:
len(df_dummies)

In [ ]:
unique_combos = df_dummies[feat_cols].drop_duplicates()
print(f"Number of unique combinations in df_dummies: {unique_combos.shape[0]}")

### Randomly generate crash=0 combos up to len(unique_combos)

In [ ]:
def generate_random_combos(num_combos, num_features):
    return np.random.randint(2, size=(num_combos, num_features))

In [ ]:
# Number of new combinations to generate
num_new_combos = len(unique_combos)

In [ ]:
# Generate random combinations
random_combos = generate_random_combos(num_new_combos, len(feat_cols))
random_combos_df = pd.DataFrame(random_combos, columns=feat_cols)

In [ ]:
# Check for uniqueness
unique_random_combos = random_combos_df.drop_duplicates()
print(f"Number of unique random combinations generated: {unique_random_combos.shape[0]}")

### Ensure New Combos are Unique

In [ ]:
merged_combos = unique_random_combos.merge(unique_combos, how='left', indicator=True)
missing_combos = merged_combos[merged_combos['_merge'] == 'left_only'].drop('_merge', axis=1)
print(f"Number of valid new combinations: {missing_combos.shape[0]}")

### Set crash=0 for missing_combos

In [ ]:
missing_combos['crash'] = 0
missing_combos['crash'].value_counts()

### resample missing_combos before merging to balance the classes
With replacement

**Reasoning**: actual data has >400k unique crash locations with enough duplicates (or more) to have >800k observatons. A random sample of non-crash location combos then resampliing to balance the classes would better reflect reality than simply assuming no crashes will ever occur for all combos that aren't present in the data already

In [ ]:
missing_combos = missing_combos.sample(n=len(df_dummies), random_state=9, replace=True)

In [ ]:
print(len(df_dummies))
print(len(missing_combos))

### Merge data

In [ ]:
df_balanced = pd.concat([df_dummies, missing_combos], ignore_index=True)
print(f"Shape of the extended DataFrame: {df_balanced.shape}")

# Verify the distribution of 'crash' values
print(df_balanced['crash'].value_counts())

## Train & Test data split

In [ ]:
X = df_balanced.drop(columns=['crash'])
y = df_balanced['crash']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")
print()
print(f'80% of total rows: {len(X) * 0.8}')
print(f'20% of total rows: {len(X) * 0.2}')

In [ ]:
# Verify the proportions of 'crash' values in the splits
print(f"Training set distribution:\n{y_train.value_counts(normalize=True)}")
print(f"Test set distribution:\n{y_test.value_counts(normalize=True)}")

# Models

## Logistic Regression

In [ ]:
log_reg = LogisticRegression(random_state=9)
log_reg.fit(X_train, y_train)

In [ ]:
y_pred_log_reg = log_reg.predict(X_test)

In [ ]:
print("Logistic Regression Classification Report:\n", classification_report(y_test, y_pred_log_reg))

print('confusion_matrix:\n', confusion_matrix(y_test, y_pred_log_reg))

print()
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))

## Random Forest

In [ ]:
rf_clf = RandomForestClassifier(random_state=9)
rf_clf.fit(X_train, y_train)

In [ ]:
y_pred_rf = rf_clf.predict(X_test)

In [ ]:
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

print('confusion_matrix:\n', confusion_matrix(y_test, y_pred_rf))

print()
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

**Note: Not worth the time to plot ROC curve or Precision-Recall Curve since all scores essentially 1**

## Feature Importances 

### from Logistic Regression
extract the feature coefficients

In [ ]:
log_reg_feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': log_reg.coef_[0]
}).sort_values(by='importance', ascending=False)

In [ ]:
print("Top 10 Logistic Regression Feature Importances:\n", log_reg_feature_importances)

In [ ]:
# Sort the absolute value of the coefficients
log_reg_feature_importances['abs_importance'] = log_reg_feature_importances['importance'].abs()
log_reg_feature_importances = log_reg_feature_importances.sort_values(by='abs_importance', ascending=False)

# Selecting the top 10 features by magnitude
top_10_log_reg_features = log_reg_feature_importances.head(10).drop(columns=['abs_importance'])

print("Top 10 Logistic Regression Feature Importances:\n", top_10_log_reg_features)

In [ ]:
print("Logistic Regression Feature Importances:\n", log_reg_feature_importances)

### from Random Forest

In [ ]:
rf_feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_clf.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [ ]:
print("Top 10 Random Forest Feature Importances:\n", rf_feature_importances.head(10))

Quote about feature importances from both models:

    "Logistic Regression highlights features that have the strongest linear relationship with the target,
    while Random Forest emphasizes features that are most effective at splitting the data to reduce impurity, potentially capturing more complex patterns and interactions."

In [ ]:
threshold = 0.01  # Adjust this threshold as necessary
filtered_importances = rf_feature_importances[rf_feature_importances['Importance'] > threshold]

plt.figure(figsize=(8, 6))
plt.title("Random Forest Feature Importances")
plt.barh(filtered_importances['Feature'], filtered_importances['Importance'], align="center")
plt.xlabel('Importance')
plt.ylabel('Features')
plt.xlim(0, 0.10)#np.max(filtered_importances['Importance']) + 0.05)  # set to 0.10 for better visualization
plt.gca().invert_yaxis()
plt.show()

## Neural Network

In [ ]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)

In [ ]:
def create_nn_model(input_dim):
    nn_model = Sequential()
    nn_model.add(Input(shape=(input_dim,)))
    nn_model.add(Dense(64, activation='relu'))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(32, activation='relu'))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(1, activation='sigmoid'))
    
    nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return nn_model

In [ ]:
input_dim = X_train.shape[1]
nn_model = create_nn_model(input_dim)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = nn_model.fit(X_train, y_train, 
                       epochs=50, 
                       batch_size=32, 
                       validation_split=0.2, 
                       callbacks=[early_stopping],
                       verbose=1)

In [ ]:
y_pred_proba = nn_model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

In [ ]:
# Classification report and confusion matrix
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()